In [1]:
import numpy as np
import time
import os
from Frozen_Lake import FrozenLakeEnv

env = FrozenLakeEnv()

Если награда не зависит от $s'$:
$$
q(s,a) = R(s, a) + \gamma \sum_{s'} P(s'|s,a) v(s')
$$
Если награда зависит от $s'$:
$$
q(s,a) = \sum_{s'} P(s'|s,a) \Big( R(s,a,s') + \gamma v(s')\Big)
$$
Если награда не зависит от $s'$ - это частный случай того, когда награда зависит от $s'$:
$$
q(s,a) = \sum_{s'} P(s'|s,a) \Big( R(s,a) + \gamma v(s')\Big) = R(s,a) \sum_{s'} P(s'|s,a) + \gamma \sum_{s'} P(s'|s,a) v(s')
 = R(s, a) + \gamma \sum_{s'} P(s'|s,a) v(s')
$$



In [2]:
def get_q_values(v_values, gamma):
    q_values = {}
    for state in env.get_all_states():
        q_values[state] = {}
        for action in env.get_possible_actions(state):
            q_values[state][action] = 0
            for next_state in env.get_next_states(state, action):
                q_values[state][action] += env.get_transition_prob(state, action, next_state) * env.get_reward(state, action, next_state)
                q_values[state][action] += gamma * env.get_transition_prob(state, action, next_state) * v_values[next_state]
    return q_values

In [3]:
def init_policy():
    policy = {}
    for state in env.get_all_states():
        policy[state] = {}
        for action in env.get_possible_actions(state):
            policy[state][action] = 1 / len(env.get_possible_actions(state))
    return policy

In [4]:
def init_v_values():
    v_values = {}
    for state in env.get_all_states():
        v_values[state] = 0
    return v_values

init_v_values()

{(0, 0): 0,
 (0, 1): 0,
 (0, 2): 0,
 (0, 3): 0,
 (1, 0): 0,
 (1, 1): 0,
 (1, 2): 0,
 (1, 3): 0,
 (2, 0): 0,
 (2, 1): 0,
 (2, 2): 0,
 (2, 3): 0,
 (3, 0): 0,
 (3, 1): 0,
 (3, 2): 0,
 (3, 3): 0}

In [5]:
def policy_evaluation_step(v_values, policy, gamma):
    q_values = get_q_values(v_values, gamma)
    new_v_values = init_v_values()
    for state in env.get_all_states():
        new_v_values[state] = 0
        for action in env.get_possible_actions(state):
            new_v_values[state] += policy[state][action] * q_values[state][action]
    return new_v_values

In [19]:
def policy_evaluation(policy, gamma, eval_iter_n):
    v_values = init_v_values()
    new_v_values = policy_evaluation_step(v_values, policy, gamma)
    delta_v = abs(np.array(list(v_values.values())) - np.array(list(new_v_values.values())))

    eps = 0.01
    while(max(delta_v) > eps):
        v_values = new_v_values
        new_v_values = policy_evaluation_step(v_values, policy, gamma)
        delta_v = abs(np.array(list(v_values.values())) - np.array(list(new_v_values.values())))
        
    q_values = get_q_values(v_values, gamma)
    return q_values

In [17]:
def policy_improvement(q_values):
    policy = {}
    for state in env.get_all_states():
        policy[state] = {}
        argmax_action = None
        max_q_value = float('-inf')
        for action in env.get_possible_actions(state): 
            policy[state][action] = 0
            if q_values[state][action] > max_q_value:
                argmax_action = action
                max_q_value = q_values[state][action]
        policy[state][argmax_action] = 1
    return policy

Тренировка: evaluation + improvement

In [20]:
iter_n = 1000
eval_iter_n = 100
gamma = 0.999

policy = init_policy()
for _ in range(iter_n):
    q_values = policy_evaluation(policy, gamma, eval_iter_n)
    policy = policy_improvement(q_values)

In [9]:
policy_evaluation(policy, gamma, eval_iter_n)

{(0, 0): {'left': 0.8457375418648228,
  'down': 0.8463471369266533,
  'right': 0.830179726547895,
  'up': 0.8434279118107144},
 (0, 1): {'left': 0.7588906479381814,
  'down': 0.1662921229228293,
  'right': 0.736631409360287,
  'up': 0.8270324185861658},
 (0, 2): {'left': 0.8186337759153014,
  'down': 0.7720262121949417,
  'right': 0.8004911252413724,
  'up': 0.816956114983963},
 (0, 3): {'left': 0.734363578026046,
  'down': 0.16207956467397222,
  'right': 0.7229223506130836,
  'up': 0.8046772096633799},
 (1, 0): {'left': 0.8499713256486713,
  'down': 0.7746359724515057,
  'right': 0.17075398955246807,
  'up': 0.7612002779922897},
 (1, 1): {},
 (1, 2): {'left': 0.1770235884738408,
  'down': 0.7621498353883566,
  'right': 0.1770235884738408,
  'up': 0.65403887240237},
 (1, 3): {},
 (2, 0): {'left': 0.7746359724515057,
  'down': 0.183484261021784,
  'right': 0.863042449746817,
  'up': 0.8627015971179872},
 (2, 1): {'left': 0.7887544775156872,
  'down': 0.973650831713135,
  'right': 0.8611

In [21]:
total_rewards = []

for _ in range(5000):
    total_reward = 0
    state = env.reset()
    for _ in range(1000):
        action = np.random.choice(env.get_possible_actions(state), p=list(policy[state].values()))
        state, reward, done, _ = env.step(action)
        total_reward += reward
        
        if done:
            break
    
    total_rewards.append(total_reward)

np.mean(total_rewards)

0.8514

In [11]:
policy

{(0, 0): {'left': 0, 'down': 1, 'right': 0, 'up': 0},
 (0, 1): {'left': 0, 'down': 0, 'right': 0, 'up': 1},
 (0, 2): {'left': 1, 'down': 0, 'right': 0, 'up': 0},
 (0, 3): {'left': 0, 'down': 0, 'right': 0, 'up': 1},
 (1, 0): {'left': 1, 'down': 0, 'right': 0, 'up': 0},
 (1, 1): {None: 1},
 (1, 2): {'left': 0, 'down': 1, 'right': 0, 'up': 0},
 (1, 3): {None: 1},
 (2, 0): {'left': 0, 'down': 0, 'right': 1, 'up': 0},
 (2, 1): {'left': 0, 'down': 1, 'right': 0, 'up': 0},
 (2, 2): {'left': 1, 'down': 0, 'right': 0, 'up': 0},
 (2, 3): {None: 1},
 (3, 0): {None: 1},
 (3, 1): {'left': 0, 'down': 0, 'right': 1, 'up': 0},
 (3, 2): {'left': 0, 'down': 0, 'right': 1, 'up': 0},
 (3, 3): {None: 1}}

In [12]:
from IPython.display import clear_output

In [13]:
state = env.reset()
for _ in range(1000):
    action = np.random.choice(env.get_possible_actions(state), p=list(policy[state].values()))
    state, reward, done, _ = env.step(action)
    total_reward += reward

    clear_output(wait=True)
    env.render()
    time.sleep(1)
    
    if done:
        break

SFFF
*HFH
FFFH
HFFG



KeyboardInterrupt: 